# Dashboards interactivos con Atoti

En este notebook aprenderemos a crear dashboards interactivos utilizando Atoti y nuestro esquema universitario.

## 1. Configuración inicial

Importamos las bibliotecas necesarias e iniciamos una sesión:

In [ ]:
import atoti as tt
import pandas as pd
import numpy as np

session = tt.Session.start()

## 2. Crear datos de ejemplo complejos

Vamos a crear un conjunto de datos más complejo que incluya información sobre matrículas, rendimiento, docencia y movilidad:

In [ ]:
# Datos de matrículas y rendimiento
data = {
    'CursoAcademico': ['2023/2024'] * 36 + ['2022/2023'] * 36,
    'Centro': ['Facultad de Ciencias'] * 18 + ['Facultad de Letras'] * 18 + ['Facultad de Ciencias'] * 18 + ['Facultad de Letras'] * 18,
    'PlanEstudio': ['Grado en Informática', 'Grado en Matemáticas', 'Grado en Física'] * 6 + ['Grado en Historia', 'Grado en Filología', 'Grado en Filosofía'] * 6,
    'Asignatura': ['Programación', 'Cálculo', 'Física I', 'Historia Antigua', 'Literatura', 'Ética'] * 12,
    'Semestre': ['Primero', 'Segundo'] * 36,
    'NumeroMatriculados': [45, 40, 35, 30, 25, 20] * 12,
    'CreditosAsignatura': [6, 6, 6, 6, 6, 6] * 12,
    'NotaMedia': [7.5, 7.2, 7.0, 8.0, 7.8, 7.5] * 12,
    'TasaAprobados': [0.85, 0.82, 0.80, 0.90, 0.88, 0.85] * 12,
    'HorasDocencia': [60, 60, 60, 60, 60, 60] * 12,
    'NumeroProfesores': [2, 2, 2, 2, 2, 2] * 12,
    'EstudiantesMovilidad': [5, 4, 3, 2, 1, 1] * 12,
    'ProfesoresMovilidad': [1, 1, 1, 0, 0, 0] * 12
}

df = pd.DataFrame(data)
df

## 3. Crear tabla y cubo

Cargamos los datos en una tabla de Atoti y creamos un cubo:

In [ ]:
# Crear tabla
academico_table = session.read_pandas(
    df,
    table_name="Academico",
    keys=["CursoAcademico", "Centro", "PlanEstudio", "Asignatura", "Semestre"]
)

# Crear cubo
cube = session.create_cube(academico_table)

## 4. Crear medidas calculadas para el dashboard

Vamos a crear algunas medidas calculadas útiles para nuestro dashboard:

In [ ]:
# Medida para la eficiencia docente
cube.measures["Eficiencia Docente"] = tt.agg.mean(
    cube.measures["NotaMedia.MEAN"] / (cube.measures["HorasDocencia.SUM"] / cube.measures["NumeroMatriculados.SUM"])
)

# Medida para la carga docente por profesor
cube.measures["Carga Docente por Profesor"] = tt.agg.sum(
    cube.measures["HorasDocencia.SUM"] / cube.measures["NumeroProfesores.SUM"]
)

# Medida para la tasa de éxito ponderada
cube.measures["Tasa Exito Ponderada"] = tt.agg.mean(
    cube.measures["TasaAprobados.MEAN"] * cube.measures["NotaMedia.MEAN"] / 10
)

# Medida para la internacionalización
cube.measures["Indice Internacionalizacion"] = tt.agg.mean(
    (cube.measures["EstudiantesMovilidad.SUM"] + cube.measures["ProfesoresMovilidad.SUM"]) / 
    (cube.measures["NumeroMatriculados.SUM"] + cube.measures["NumeroProfesores.SUM"])
)

## 5. Crear dashboard interactivo

Vamos a crear un dashboard interactivo que muestre diferentes aspectos de los datos:

In [ ]:
# Crear widget interactivo
widget = session.widget
widget

## 6. Crear visualizaciones específicas

Vamos a crear algunas visualizaciones específicas para nuestro dashboard:

In [ ]:
# 1. Gráfico de rendimiento por centro
cube.query(
    [cube.measures["Tasa Exito Ponderada"], cube.measures["Eficiencia Docente"]],
    levels=[cube.hierarchies["Centro"]],
    filter=cube.hierarchies["CursoAcademico"] == "2023/2024"
)

In [ ]:
# 2. Análisis de movilidad
cube.query(
    [cube.measures["Indice Internacionalizacion"], cube.measures["EstudiantesMovilidad.SUM"]],
    levels=[cube.hierarchies["Centro"], cube.hierarchies["PlanEstudio"]],
    filter=cube.hierarchies["CursoAcademico"] == "2023/2024"
)

In [ ]:
# 3. Análisis de carga docente
cube.query(
    [cube.measures["Carga Docente por Profesor"], cube.measures["NumeroProfesores.SUM"]],
    levels=[cube.hierarchies["Centro"], cube.hierarchies["PlanEstudio"]],
    filter=cube.hierarchies["CursoAcademico"] == "2023/2024"
)

## 7. Crear dashboard personalizado

Vamos a crear un dashboard personalizado con múltiples visualizaciones:

In [ ]:
# Crear dashboard personalizado
dashboard = session.dashboards.create(
    "Dashboard Academico",
    "Dashboard para análisis académico"
)

# Añadir visualizaciones al dashboard
dashboard.add_chart(
    "Rendimiento por Centro",
    cube.query(
        [cube.measures["Tasa Exito Ponderada"], cube.measures["Eficiencia Docente"]],
        levels=[cube.hierarchies["Centro"]],
        filter=cube.hierarchies["CursoAcademico"] == "2023/2024"
    )
)

dashboard.add_chart(
    "Análisis de Movilidad",
    cube.query(
        [cube.measures["Indice Internacionalizacion"], cube.measures["EstudiantesMovilidad.SUM"]],
        levels=[cube.hierarchies["Centro"], cube.hierarchies["PlanEstudio"]],
        filter=cube.hierarchies["CursoAcademico"] == "2023/2024"
    )
)

dashboard.add_chart(
    "Carga Docente",
    cube.query(
        [cube.measures["Carga Docente por Profesor"], cube.measures["NumeroProfesores.SUM"]],
        levels=[cube.hierarchies["Centro"], cube.hierarchies["PlanEstudio"]],
        filter=cube.hierarchies["CursoAcademico"] == "2023/2024"
    )
)

# Mostrar dashboard
dashboard

## Ejercicios Prácticos

1. Crea un dashboard que muestre la evolución temporal de los indicadores clave
2. Implementa un dashboard que compare el rendimiento entre diferentes planes de estudio
3. Crea un dashboard que analice la relación entre movilidad y rendimiento académico
4. Implementa un dashboard que muestre la eficiencia docente por asignatura